In [40]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
from bs4 import BeautifulSoup

serv_obj = Service("C:\chromedriver_win32\chromedriver.exe")
driver = webdriver.Chrome(service=serv_obj)
driver.get("https://www.flipkart.com/")
driver.maximize_window()
time.sleep(1)



driver.implicitly_wait(2)
time.sleep(1)

#Close the cookie window by accepting
driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _2doB4z"]').click()
time.sleep(1)

#click the search area
fin = driver.find_element(By.XPATH,'//input[@class="_3704LK"]')
fin.click()
time.sleep(1)

#search for laptos
fin.send_keys('laptops')
time.sleep(1)

#clicck the search button
driver.find_element(By.XPATH,'//button[@class="L0Z3Pu"]').click()

url = "https://www.flipkart.com/search?q=laptops&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off"
req = requests.get(url).text
scraper = BeautifulSoup(req)

#find the container tofind the old price of the product
containers = scraper.find_all('div',{'class': '_25b18c'})
#divi = scraper.find_all('div',{'class': '_3I9_wc _27UcVY'})
#<div class="_3I9_wc _27UcVY">₹75,799</div>
divisions = scraper.find_all('div',{'class': '_3I9_wc _27UcVY'})

#setting the dataframe coloums
df = pd.DataFrame(columns=['product' , 'price' , 'old_price' , 'Description'])

pro = []
price = []
product = []
ram1 = []
opersys = []
ssd1 = []
dis = []
war = []
old = []
des = []

#loop through thepages till page=3
for k in range(2 , 4):
   

    #Extracting the product name
    titles = driver.find_elements(By.XPATH,'//div[@class="_4rR01T"]')
    for title in titles:
        product.append(title.text)
    
    #Extractring the price of the product
    prices = driver.find_elements(By.XPATH,'//div[@class="_30jeq3 _1_WHN1"]')
    for price1 in prices:
        pr = price1.text
        pr = pr.replace(pr[0], '')    
        price.append(pr)
      
    #Extracting the old price of the product
    for n in range(len(divisions)):
        try:
            if divisions[n]!=None:
                ol = divisions[n].text
                ol = ol.replace(ol[0], '') 
                old.append(ol)
        except:
            old.append("")    
    
 
       
    #finding the description of the product
    processors = driver.find_elements(By.XPATH,'//ul[@class="_1xgFaf"]/descendant::li[1]')
    RAM = driver.find_elements(By.XPATH,'//ul[@class="_1xgFaf"]/descendant::li[2]')
    opsys = driver.find_elements(By.XPATH,'//ul[@class="_1xgFaf"]/descendant::li[3]')
    SSD = driver.find_elements(By.XPATH,'//ul[@class="_1xgFaf"]/descendant::li[4]')
    displays = driver.find_elements(By.XPATH,'//ul[@class="_1xgFaf"]/descendant::li[5]')
    warrantys = driver.find_elements(By.XPATH,'//ul[@class="_1xgFaf"]/descendant::li[6]')
    
    
    for processor in processors:
        pro.append(processor.text)
    
    for ram in RAM:
        ram1.append(ram.text)
        
    for oper in opsys:
        opersys.append(oper.text)
    for ssd in SSD:
        ssd1.append(ssd.text)
    for display in displays:
        dis.append(display.text)
    for warranty in warrantys:
        war.append(warranty.text)
        
    #putting all the description in one array    
    for l in range(len(processors)):
        des.append(pro[l] + " " + ram1[l] +" "+ opersys[l] +" "+ssd1[l] +" " +dis[l]+" "+war[l]+" " )
        
    #pagination    
    driver.get("https://www.flipkart.com/search?q=laptops&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page="+ str(k))
    

#writing the extracted data to a Data Frame

df = pd.DataFrame({'product' : product , 'price': price , 'old_price' : old , 'Description' : des})
#convert the data frame to an Excel Sheet

df.to_csv('flipkartdata.csv' , index=False)
